# Graden Groups

In [9]:
import numpy as np

## Part 1

In [1]:
def parse_input(n_sample=None, file_path="Garden_Groups.txt", mode="r"):
	if n_sample is not None:
		file_path = f"sample{n_sample}.txt"
		with open(file_path, mode) as f:
			return f.read().splitlines()

	else:
		with open(file_path, mode) as f:
			return f.read().splitlines()

In [2]:
def get_group(garden, x, y, group_type=None, group_perimeter=None, group_edge_positions=None, visited=None):
	
	directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]
	
	if visited is None:
		visited = set()
	if group_type is None:
		group_type = garden[x][y]
	if group_perimeter is None:
		group_perimeter = 0
	if group_edge_positions is None:
		group_edge_positions = set()

	if (x, y) in visited:
		return visited
	
	visited.add((x, y))
	group_edge_positions.add((x, y))

	for (dx, dy) in directions:
		nx, ny = x + dx, y + dy
		if nx < 0 or nx >= len(garden) or ny < 0 or ny >= len(garden[0]):
			group_perimeter += 1
			group_edge_positions.add((x, y))
			continue
		elif (nx, ny) in visited:
			continue
		elif garden[nx][ny] != group_type:
			group_perimeter += 1
			group_edge_positions.add((x, y))
			continue
		else:
			_, group_perimeter, group_edge_positions = get_group(garden, nx, ny, group_type, group_perimeter, group_edge_positions, visited)
		
	return visited, group_perimeter, group_edge_positions

In [3]:
def get_all_groups(garden):
	garden_dims = (len(garden), len(garden[0]))

	all_groups = []
	all_edges = []
	all_edge_positions = []

	grouped_elements = [[False for _ in range(garden_dims[1])] for _ in range(garden_dims[0])]

	for x in range(garden_dims[0]):
		for y in range(garden_dims[1]):
			if grouped_elements[x][y]:
				continue
			group, edges, edge_pos = get_group(garden, x, y)
			all_groups.append((garden[x][y], group))
			all_edges.append(edges)
			all_edge_positions.append(edge_pos)

			for (i, j) in group:
				grouped_elements[i][j] = True

	return all_groups, all_edges, all_edge_positions

In [4]:
def get_cost(groups, edges, verbose=False):
	cost = 0 
	for (group_type, group), edge in zip(groups, edges):
		if verbose:
			print(group_type, len(group), edge, len(group) * edge)
		cost += len(group) * edge

	return cost

In [5]:
def part1(sample=None, verbose=False):
	garden = parse_input(sample)
	groups, edges, edge_pos = get_all_groups(garden)
	cost = get_cost(groups, edges, verbose=verbose)
	return cost

In [6]:
part1()

1402544

## Part 2

In [10]:
def get_group_corners(group, garden_dims):
						#  Up,      Down
	vertical = np.array([(-1, 0), (1, 0)])
						#   Left,    Right 
	horizontal = np.array([(0, -1), (0, 1)])

	diagonals = np.array([(-1, -1), (-1, 1)],
					  [(1, -1), (1, 1)])

	h, v, c = 0, 0, 0
	h_idx, v_idx = None, None

	# Exterior corners
	for (x, y) in group:
		actual_plant = np.array([x, y])
		for j, v_dir in enumerate(vertical):
			if (actual_plant + v_dir) in group:
				v += 1
				v_idx = j
		
		for i, h_dir in enumerate(horizontal):
			if (actual_plant + h_dir) in group:
				h += 1
				h_idx = i		